### Dataset

In this homework, we will continue the New York City Airbnb Open Data. You can take it from
[Kaggle](https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data?select=AB_NYC_2019.csv)
or download from [here](https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv)
if you don't want to sign up to Kaggle.

We'll keep working with the `'price'` variable, and we'll transform it to a classification task.

In [300]:
from pathlib import PurePath, Path
import pandas as pd 
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

In [301]:
def download_dataset():
    dataset = "new-york-city-airbnb-open-data"
    datasets_path = "../../datasets"
    fullpath = Path(datasets_path,f"{dataset}.zip")
    
    if not fullpath.is_file():
        !kaggle datasets download -d dgomonov/$dataset -p $datasets_path
        !ls $fullpath
        !unzip $fullpath -d $datasets_path -o

In [302]:
download_dataset()

In [303]:
# Read the data 
path_to_data = "../../datasets/AB_NYC_2019.csv"
data = pd.read_csv(path_to_data)

In [304]:
data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


### Features

For the rest of the homework, you'll need to use the features from the previous homework with additional two `'neighbourhood_group'` and `'room_type'`. So the whole feature set will be set as follows:

* `'neighbourhood_group'`,
* `'room_type'`,
* `'latitude'`,
* `'longitude'`,
* `'price'`,
* `'minimum_nights'`,
* `'number_of_reviews'`,
* `'reviews_per_month'`,
* `'calculated_host_listings_count'`,
* `'availability_365'`

Select only them and fill in the missing values with 0.

In [305]:
features = [ 'neighbourhood_group', 'latitude', 'longitude', 'room_type', 'price',
             'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count',
             'availability_365']

In [306]:
df = data[features]
df.head()

,neighbourhood_group,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,Brooklyn,40.64749,-73.97237,Private room,149,1,9,0.21,6,365
1,Manhattan,40.75362,-73.98377,Entire home/apt,225,1,45,0.38,2,355
2,Manhattan,40.80902,-73.94190,Private room,150,3,0,NaN,1,365
3,Brooklyn,40.68514,-73.95976,Entire home/apt,89,1,270,4.64,1,194
4,Manhattan,40.79851,-73.94399,Entire home/apt,80,10,9,0.10,1,0


In [307]:
df.isnull().sum()

neighbourhood_group                   0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [308]:
# fill missing values
df = df.fillna(0)

In [309]:
df.isnull().sum()

neighbourhood_group               0
latitude                          0
longitude                         0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

### Question 1

What is the most frequent observation (mode) for the column `'neighbourhood_group'`?

In [310]:
df['neighbourhood_group'].describe().top

'Manhattan'

### Split the data

* Split your data in train/val/test sets, with 60%/20%/20% distribution.
* Use Scikit-Learn for that (the `train_test_split` function) and set the seed to 42.
* Make sure that the target value ('price') is not in your dataframe.

In [311]:
seed = 42 
df_full_train, df_test =  train_test_split(df, train_size=0.6,random_state=seed)
df_train, df_val = train_test_split(df_full_train, test_size=0.2,random_state=seed)

In [312]:
# Remove target variable 
y_train = df_train['price']
y_test  = df_test['price']
y_val   = df_val['price']

del df_train['price']
del df_test['price']
del df_val['price']

In [313]:
# Reset index
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

In [314]:
df_train.head(3)

,neighbourhood_group,latitude,longitude,room_type,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,Brooklyn,40.71286,-73.95648,Private room,3,0,0.00,1,365
1,Queens,40.71576,-73.86268,Entire home/apt,2,97,1.43,1,230
2,Brooklyn,40.68166,-73.94325,Entire home/apt,2,20,0.31,1,355


### Question 2

* Create the [correlation matrix](https://www.google.com/search?q=correlation+matrix) for the numerical features of your train dataset.
   * In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.
* What are the two features that have the biggest correlation in this dataset?

Example of a correlation matrix for the car price dataset:

<img src="images/correlation-matrix.png" />

In [315]:
numerical = list(df_train.dtypes[df_train.dtypes!='object'].index)
numerical

['latitude',
 'longitude',
 'minimum_nights',
 'number_of_reviews',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365']

In [316]:
# Get correlation matrix
corr_matrix = df_train[numerical].corr()
corr_matrix

,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
latitude,1.000000,0.086690,0.031609,-0.012492,-0.019288,0.020220,-0.002485
longitude,0.086690,1.000000,-0.067138,0.055343,0.136692,-0.117541,0.082467
minimum_nights,0.031609,-0.067138,1.000000,-0.082787,-0.126538,0.120989,0.139308
number_of_reviews,-0.012492,0.055343,-0.082787,1.000000,0.583910,-0.073260,0.174562
reviews_per_month,-0.019288,0.136692,-0.126538,0.583910,1.000000,-0.048592,0.167429
calculated_host_listings_count,0.020220,-0.117541,0.120989,-0.073260,-0.048592,1.000000,0.223109
availability_365,-0.002485,0.082467,0.139308,0.174562,0.167429,0.223109,1.000000


### Answer #2

The 2 features most correlated are: 
- number_of_reviews and reviews_per_month with 0.58 
- calculated_host_listings_count and availability_365 with 0.22

### Make price binary

* We need to turn the price variable from numeric into binary.
* Let's create a variable `above_average` which is `1` if the price is above (or equal to) `152`.

In [317]:
y_train_above_avg = (y_train.values >= 152).astype(int)
y_test_above_avg = (y_test.values >= 152).astype(int)
y_val_above_avg = (y_val.values >= 152).astype(int)

In [318]:
# Validate the new column against the real value
print(y_train_above_avg[:5])
y_train[:5]

[1 0 0 0 0]


29700    175
3246      85
3170     149
3017     116
16021     60
Name: price, dtype: int64

### Question 3

* Calculate the mutual information score with the (binarized) price for the two categorical variables that we have. Use the training set only.
* Which of these two variables has bigger score?
* Round it to 2 decimal digits using `round(score, 2)`

In [319]:
from sklearn.metrics import mutual_info_score

In [320]:
categorical = list(df_train.dtypes[df_train.dtypes==object].index)
categorical

['neighbourhood_group', 'room_type']

In [321]:
# Between churn and contract
for cat in categorical:
    score = mutual_info_score(y_train_above_avg, df_train[cat])
    print(f"Score between price and {cat}: {score.round(2)}")

Score between price and neighbourhood_group: 0.05
Score between price and room_type: 0.14


### Answer 3 

Room type has the bigger score. 

### Question 4

* Now let's train a logistic regression
* Remember that we have two categorical variables in the data. Include them using one-hot encoding.
* Fit the model on the training dataset.
   * To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
   * `model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)`
* Calculate the accuracy on the validation dataset and rount it to 2 decimal digits.

In [322]:
# One hot encoding
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer(sparse=False)
train_dict = df_train[categorical + numerical].to_dict(orient='records')
# Get X_train
X_train = dv.fit_transform(train_dict)

# Get X_val 
val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [323]:
dv.get_feature_names()

['availability_365',
 'calculated_host_listings_count',
 'latitude',
 'longitude',
 'minimum_nights',
 'neighbourhood_group=Bronx',
 'neighbourhood_group=Brooklyn',
 'neighbourhood_group=Manhattan',
 'neighbourhood_group=Queens',
 'neighbourhood_group=Staten Island',
 'number_of_reviews',
 'reviews_per_month',
 'room_type=Entire home/apt',
 'room_type=Private room',
 'room_type=Shared room']

In [324]:
# Training the model 
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
model.fit(X_train, y_train_above_avg)

LogisticRegression(random_state=42)

In [325]:
model.intercept_[0]

-0.002445387122932676

In [326]:
model.coef_

array([[ 0.00319479,  0.00519523, -0.21866832, -0.08922699, -0.00997329,
        -0.35468284,  0.12569755,  1.14716032, -0.8021554 , -0.11845967,
        -0.00271842, -0.0673488 ,  1.72121705, -1.29844574, -0.42521136]])

In [327]:
# Get accuraccy 

# Get the predictions on validation set
y_pred = model.predict_proba(X_val)
y_pred

array([[0.47495395, 0.52504605],
       [0.9437828 , 0.0562172 ],
       [0.38300922, 0.61699078],
       ...,
       [0.9788833 , 0.0211167 ],
       [0.61937829, 0.38062171],
       [0.15232948, 0.84767052]])

In [328]:
# Get positive class only 
y_pred = y_pred[:,1]
y_pred

array([0.52504605, 0.0562172 , 0.61699078, ..., 0.0211167 , 0.38062171,
       0.84767052])

In [329]:
price_decision = (y_pred >= 0.5 )

In [330]:
(y_val_above_avg == price_decision).mean()

0.787321063394683

In [331]:
df_pred = pd.DataFrame()
df_pred['probability'] = y_pred
df_pred['prediction'] = price_decision.astype(int)
df_pred['actual'] = y_val_above_avg
df_pred['correct'] = df_pred.prediction == df_pred.actual

In [359]:
acc = round(df_pred.correct.mean(),4)
print(f"Accuracy on validation set: {acc*100}%")

Accuracy on validation set: 78.73%


### Question 5

* We have 9 features: 7 numerical features and 2 categorical.
* Let's find the least useful one using the *feature elimination* technique.
* Train a model with all these features (using the same parameters as in Q4).
* Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
* For each feature, calculate the difference between the original accuracy and the accuracy without the feature. 
* Which of following feature has the smallest difference? 
   * `neighbourhood_group`
   * `room_type` 
   * `number_of_reviews`
   * `reviews_per_month`

> **note**: the difference doesn't have to be positive

In [352]:
def one_hot_dict_vectorizer_exclude(exclude=None):

    df_train_all = df_train[categorical + numerical].drop(exclude, axis=1) 
    dv = DictVectorizer(sparse=False)
    train_dict = df_train_all.to_dict(orient='records')
    # Get X_train
    X_train = dv.fit_transform(train_dict)
    
    # Get X_val 
    df_val_all = df_val[categorical + numerical].drop(exclude, axis=1) 
    val_dict = df_val_all.to_dict(orient='records')
    X_val = dv.transform(val_dict)  
    
    return X_train, X_val

In [354]:
def train_model(X_train, X_val):
    model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
    model.fit(X_train, y_train_above_avg)
    y_pred = model.predict_proba(X_val)[:,1]
    
    df_pred = pd.DataFrame()
    df_pred['probability'] = y_pred
    price_decision = (y_pred >= 0.5 )
    df_pred['prediction'] = price_decision.astype(int)
    df_pred['actual'] = y_val_above_avg
    df_pred['correct'] = df_pred.prediction == df_pred.actual
    acc = round(df_pred.correct.mean(),4)
    return acc

In [355]:
import warnings
warnings.filterwarnings('ignore')

In [356]:
all_features = categorical + numerical
accuracies = {}
for feature in all_features:
    x_train, x_val = one_hot_dict_vectorizer_exclude(exclude=feature)
    accuracies[feature] = train_model(x_train, x_val)

In [360]:
accuracies

{'neighbourhood_group': 0.7469,
 'room_type': 0.721,
 'latitude': 0.7899,
 'longitude': 0.7889,
 'minimum_nights': 0.7865,
 'number_of_reviews': 0.788,
 'reviews_per_month': 0.788,
 'calculated_host_listings_count': 0.7872,
 'availability_365': 0.7872}

In [361]:
differences = {}
for key in ['neighbourhood_group','room_type','number_of_reviews','reviews_per_month']:
    differences[key] = acc - accuracies[key]
print(differences)

{'neighbourhood_group': 0.04039999999999999, 'room_type': 0.06630000000000003, 'number_of_reviews': -0.0007000000000000339, 'reviews_per_month': -0.0007000000000000339}


### Question 6

* For this question, we'll see how to use a linear regression model from Scikit-Learn
* We'll need to use the original column `'price'`. Apply the logarithmic transformation to this column.
* Fit the Ridge regression model on the training data.
* This model has a parameter `alpha`. Let's try the following values: `[0, 0.01, 0.1, 1, 10]`
* Which of these alphas leads to the best RMSE on the validation set? Round your RMSE scores to 3 decimal digits.

If there are multiple options, select the smallest `alpha`.

In [362]:
# Training the model 
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [363]:
def one_hot_dict_vectorizer():

    dv = DictVectorizer(sparse=False)
    train_dict = df_train[categorical + numerical].to_dict(orient='records')
    # Get X_train
    X_train = dv.fit_transform(train_dict)
    
    # Get X_val 
    val_dict = df_val[categorical + numerical].to_dict(orient='records')
    X_val = dv.transform(val_dict)  
    
    return X_train, X_val

In [364]:
def train_model_ridge(X_train, X_val, alpha):
    model = Ridge(random_state=42, alpha=alpha)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    
    rmse = mean_squared_error(y_val, y_pred,squared=False)
    return round(rmse,3)

In [365]:
x_train, x_val = one_hot_dict_vectorizer()
rmses = {}
for alpha in [0, 0.01, 0.1, 1, 10]:
    rmses[alpha] = train_model_ridge(x_train, x_val, alpha)

In [366]:
rmses

{0: 237.725, 0.01: 237.737, 0.1: 237.736, 1: 237.727, 10: 237.723}